# C1(Server) Generate streams 

This application feed data from files into a stream


In [1]:
#read data from csv
import csv
from pprint import pprint
FireData = open("FireData-Part2.csv","r")
reader = csv.reader(FireData)
FireData=[]
for item in reader:
    FireData.append(item)
    
ClimateData = open("ClimateData-Part2.csv","r")
reader = csv.reader(ClimateData)
ClimateData=[]
for item in reader:
    ClimateData.append(item)
# delete the unnecessary space
for item in ClimateData[1:]:
    item[6]=item[6].strip()
    item[7]=item[7].strip()
    item[8]=item[8].strip()

In [2]:
# change to dicitionary format
ClimatePart2=[]
for entry in ClimateData[1:]:
    dic={}
    dic['Datatype']=entry[0]
    dic['Station']=entry[1]
    dic['Air Temperature(Celcius)']=entry[2]
    dic['Relative Humidity']=entry[3]
    dic['WindSpeed  (knots)']=entry[4]
    dic['Max Wind Speed']=entry[5]
    dic['MAX']=entry[6]
    dic['MIN']=entry[7]
    dic['Precipitation']=entry[8]
    dic['FireData']=[]
    dic['Date']=''
    ClimatePart2.append(dic) 

In [3]:
# change to dicitionary format
FirePart2=[]
for item in FireData[1:]:
            dic1={}
            dic1['dtype']=item[0]
            dic1['Latitude']=item[1]
            dic1['Longitude']=item[2]
            dic1['Surface Temperature (kelvin)']=item[3]
            dic1['Power']=item[4]
            dic1['Confidence']=item[5]
            dic1['Surface Temperature (Celcius)']=item[6]
            dic1['Datetime']=''
            FirePart2.append(dic1)

In [4]:
from datetime import datetime
cdate=datetime.now().strftime('%Y-%m-%d')
ctime=datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

In [5]:
# join climate and fire, add date content and return a string

def combine_fire_climate(climate,fire):
    climate['Date']=cdate
    for item in fire:
        item['Datetime']=ctime
        climate['FireData'].append(item)
    
    result=str(climate)
    return result   

In [6]:
import socket

def send_to_stream(string):
   
    host= 'localhost'
    port=9999
    
    #(domain,type)
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    
    #  We add this line to avoid an error : "Address already in use".
    s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    
    # bind() : assigne a specific address to socket
    s.bind((host,port))
    
    # Listen to the socket. If the client calls connect(0) for a connection request,the server will receive the request
    s.listen(1)
    
    # accept the connection from client
    conn,addr=s.accept()
    
    # Encode the string into bytes and send the data.
    # utf_8 : 8-bit unicode transformation format
    conn.sendall(string.encode('utf_8'))
    

In [ ]:
# send the processed text to the pyspark client.
def connect(numberOfC,numberOfF):
    """
    
    numberOfC -- A numeric value indicates how many documents should be sent.`all` for all documents. 
    numberOfF -- A numeric value indicates how many fire documents combines with climate data.
                 (example: one tuple from climate and five tuple from fire)
    
    """
    
    print('Begin')
    if numberOfC == 'all':
        numberOfC = len(ClimatePart2)
        #print(numberOfClimate)
    new_fire=[]
    
    for i in range(0,len(FirePart2),numberOfF):
        new_fire.append(FirePart2[i:i+numberOfF])
    
    print('Waiting for connection...\n')
    for index in range(0,numberOfC):
        # print(numberOfC) 366 
        c=ClimatePart2[index]
        f=new_fire[index]
        to_be_sent=combine_fire_climate(c,f)
        send_to_stream(to_be_sent)
        print('file',index+1,' already sent')
    print('Finish')


In [ ]:
connect('all',4)

Begin
Waiting for connection...

file 1  already sent
file 2  already sent
file 3  already sent
file 4  already sent
file 5  already sent
